In [92]:
# import tensorflow as tf
import time
import numpy as np
import os
import copy
import pickle
import argparse
import utility
import pandas as pd
from sklearn.metrics import *



In [93]:
import pandas as pd


In [107]:
train_df = pd.read_pickle(r'Yelp-4/training_df.pkl')    
vali_df = pd.read_pickle(r'Yelp-4/valiing_df.pkl')   # for validation
testing_df = pd.read_pickle(r'Yelp-4/testing_df.pkl')     # for testing
key_genre = pd.read_pickle(r'Yelp-4/key_genre.pkl')  
item_idd_genre_list = pd.read_pickle(r'Yelp-4/item_idd_genre_list.pkl')   
genre_item_vector = pd.read_pickle(r'Yelp-4/genre_item_vector.pkl')    
genre_count = pd.read_pickle(r'Yelp-4/genre_count.pkl')      
user_genre_count = pd.read_pickle(r'Yelp-4/user_genre_count.pkl') 

num_item = len(train_df['item_id'].unique())
num_user = len(train_df['user_id'].unique())
num_genre = len(key_genre)

item_genre_list = []
for u in range(num_item):
    gl = item_idd_genre_list[u]
    tmp = []
    for g in gl:
        if g in key_genre:
            tmp.append(g)
    item_genre_list.append(tmp)

In [108]:
item_genre = np.zeros((num_item, num_genre))
for i in range(num_item):
    gl = item_genre_list[i]
    for k in range(num_genre):
        if key_genre[k] in gl:
            item_genre[i, k] = 1.0

In [109]:
item_genre

array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [1., 1., 1., 0.],
       ...,
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.]])

In [110]:
# train_df = train_df[train_df['user_id'].isin(vali_df['user_id'].unique())]
# train_df = train_df[train_df['item_id'].isin(vali_df['item_id'].unique())]

In [111]:
def create_user_list(df, user_size):
    user_list = [list() for u in range(user_size)]
    for row in df.itertuples():
        user_list[row.user_id].append(row.item_id)
    return user_list

def create_pair(user_list):
    pair = []
    for user, item_list in enumerate(user_list):
        pair.extend([(user, item) for item in item_list])
    return pair

In [112]:
df_shuffled = testing_df.sample(frac=1)
df_splits = np.array_split(df_shuffled, 2)


In [113]:
train_user_list = create_user_list(df_splits[0], num_user)
test_user_list = create_user_list(df_splits[1], num_user)
train_pair = create_pair(train_user_list)


In [114]:
num_sample, user_list, item_pos_list, item_neg_list = utility.negative_sample(testing_df, num_user,
                                                                                      num_item, 5)

In [115]:
pair = []
pair.extend([(int(user), int(item)) for user, item in zip(user_list,item_pos_list)])

In [116]:
len(pair)

101625

In [117]:
len(train_pair)

10163

In [118]:
dataset = {'user_size': num_user, 'item_size': num_item,
            'train_user_list': train_user_list, 'test_user_list': test_user_list,
            'train_pair': pair}

In [119]:
dirname = os.path.dirname(os.path.abspath('preprocessed/yelp-4-tune-data.pickle'))
os.makedirs(dirname, exist_ok=True)
with open('preprocessed/yelp-4-tune-data', 'wb') as f:
    pickle.dump(dataset, f, protocol=pickle.HIGHEST_PROTOCOL)